<a href="https://colab.research.google.com/github/esc-edson/monitorIA/blob/main/MonitorIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Código final
!pip -q install google-genai gspread google-auth oauth2client sib-api-v3-sdk

import os
from google.colab import userdata
from google import genai
import gspread
from google.oauth2.service_account import Credentials
import smtplib
from email.mime.text import MIMEText
import json
import sib_api_v3_sdk
from sib_api_v3_sdk import ApiClient, Configuration
from sib_api_v3_sdk.api import transactional_emails_api
from sib_api_v3_sdk.models import *

# Carregar a chave da API do Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])
modelo_nome = "gemini-2.0-flash"

# --- Configuração do Google Sheets ---
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive.file'
]
credentials_info = json.loads(userdata.get('GSPREAD_CREDENTIALS'))
creds = Credentials.from_service_account_info(credentials_info, scopes=scopes)
gc = gspread.authorize(creds)

# *** INSERIR O ID DA PLANILHA DO PROFISSIONAL ***
ID_PLANILHA_PROFISSIONAL = '1Y-49xnmp8wj4NMVrJ8aoKVTUiQsBvG0TVvuprI5yegA'
planilha_profissional = gc.open_by_key(ID_PLANILHA_PROFISSIONAL)
aba_configuracoes_profissional = planilha_profissional.worksheet('Configurações')
dados_configuracoes_profissional = aba_configuracoes_profissional.get_all_values()

# Assumindo que o nome do profissional esteja na segunda linha, primeira coluna (índice [1][0])
nome_profissional = dados_configuracoes_profissional[1][0]
# Assumindo que o email do profissional esteja na segunda linha, segunda coluna (índice [1][1])
email_profissional = dados_configuracoes_profissional[1][1]
# Assumindo que o tratamento desejado (Dr. ou Dra.) esteja na segunda linha, terceira coluna (índice [1][2])
tratamento_desejado = dados_configuracoes_profissional[1][2] if len(dados_configuracoes_profissional) > 1 and len(dados_configuracoes_profissional[1]) > 2 else ''
# Assumindo que o tipo de tratamento esteja na segunda linha, quarta coluna (índice [1][3])
tipo_tratamento = dados_configuracoes_profissional[1][3] if len(dados_configuracoes_profissional) > 1 and len(dados_configuracoes_profissional[1]) > 3 else ''

nome_profissional_final = f"{tratamento_desejado} {nome_profissional}" if tratamento_desejado else nome_profissional

artigo_tratamento = ""
if tratamento_desejado.lower() == "dr.":
    artigo_tratamento = "o"
elif tratamento_desejado.lower() == "dra.":
    artigo_tratamento = "a"

# Carregar as perguntas padrão da aba 'Perguntas Padrão'
aba_perguntas = planilha_profissional.worksheet('Perguntas Padrão')
perguntas_padrao = [p[0] for p in aba_perguntas.get_all_values() if p[0]]

# --- Configuração de Email (AGORA COM BREVO SDK) ---
def enviar_notificacao_email(destinatario, assunto, corpo):
    # Configure a chave de API do Brevo
    configuration = Configuration()
    configuration.api_key['api-key'] = userdata.get('BREVO_API_KEY')

    # Crie uma instância da API Client
    api_client = ApiClient(configuration)

    # Crie uma instância da API Transactional Emails
    api_instance = transactional_emails_api.TransactionalEmailsApi(api_client)

    # Defina os parâmetros do e-mail
    send_smtp_email = SendSmtpEmail(
        to=[{"email": destinatario}],
        sender={"email": "assistenteia.monitoria@gmail.com", "name": "MonitorIA"},
        subject=assunto,
        html_content=f"<p>{corpo}</p>"
    )

    try:
        # Envie o e-mail
        api_response = api_instance.send_transac_email(send_smtp_email)
        print(f"Notificação enviada para: {destinatario} (ID da mensagem: {api_response.message_id})")
    except ApiException as e:
        print(f"Erro ao enviar e-mail com Brevo SDK: {e}")

historico_atualizacoes = {}
piora_detectada = False
aumento_vermelhidao_significativo = 3
autorizacao_compartilhamento = False
primeira_conversa = True # Flag para identificar a primeira interação
enviou_foto = False # Flag para controlar se a foto já foi solicitada e (tentativamente) enviada
autorizacao_concedida = False # Nova flag para controlar o fluxo

# Palavras-chave para respostas afirmativas e negativas
palavras_afirmativas = ["sim", "claro", "afirmativo", "ok", "pode ser", "com certeza", "yes", "s"]
palavras_negativas = ["não", "negativo", "de jeito nenhum", "nunca", "tá loco", "no", "n"]

def verificar_resposta(resposta, lista_positiva, lista_negativa):
    resposta_lower = resposta.lower()
    if any(palavra in resposta_lower for palavra in lista_positiva):
        return True
    elif any(palavra in resposta_lower for palavra in lista_negativa):
        return False
    return None # Retorna None se não encontrar palavras-chave

def verificar_piora(nome_paciente, historico_atualizacoes):
    if len(historico_atualizacoes.get(nome_paciente, [])) >= 2:
        ultima_atualizacao = historico_atualizacoes[nome_paciente][-1]
        penultima_atualizacao = historico_atualizacoes[nome_paciente][-2]
        try:
            vermelhidao_atual = int(ultima_atualizacao.split("Vermelhidão: ")[1].split("/")[0]) if "Vermelhidão:" in ultima_atualizacao else -1
            vermelhidao_anterior = int(penultima_atualizacao.split("Vermelhidão: ")[1].split("/")[0]) if "Vermelhidão:" in penultima_atualizacao else -1
            if vermelhidao_atual != -1 and vermelhidao_anterior != -1 and (vermelhidao_atual - vermelhidao_anterior) >= aumento_vermelhidao_significativo:
                return True
        except (IndexError, ValueError):
            return False
    return False

def gerar_proxima_interacao(nome_paciente, historico_atualizacoes, perguntas_padrao, ultima_pergunta_monitoria):
    historico_texto = "\n".join(historico_atualizacoes.get(nome_paciente, []))
    ultima_atualizacao_paciente = historico_atualizacoes.get(nome_paciente, [])[-1] if historico_atualizacoes.get(nome_paciente) else "Nenhuma atualização."

    prompt = f"""Você é um assistente virtual d{'a' if tratamento_desejado.lower() == 'dra.' else 'o'} {nome_profissional_final}, acompanhando o paciente {nome_paciente} para o tratamento {tipo_tratamento}.

**Histórico completo da conversa:**
{historico_texto}

**ÚLTIMA INTERAÇÃO:**
**Assistente:** '{ultima_pergunta_monitoria.replace('MonitorIA: ', '')}'
**Paciente:** '{ultima_atualizacao_paciente}'

Com base no histórico completo e, **ACIMA DE TUDO, NA ÚLTIMA RESPOSTA DO PACIENTE**, gere UMA PRÓXIMA PERGUNTA RELEVANTE E CONCISA.

**INSTRUÇÕES ESSENCIAIS E INEGOCIÁVEIS:**

* **SE O PACIENTE REPORTAR UMA MELHORA EM QUALQUER SINTOMA, RECONHEÇA IMEDIATAMENTE ESSA MELHORA de forma clara e positiva NA PRÓXIMA PERGUNTA.** Exemplos: 'Que ótimo que você notou uma melhora na coceira!', 'Fico feliz em saber que a vermelhidão melhorou.', 'Considerando que a coceira está melhor, ...'
* Após reconhecer a melhora (se houver), a pergunta deve buscar obter MAIS DETALHES sobre a condição atual da pele, a evolução geral ou outros sintomas.
* **NÃO FAÇA PERGUNTAS QUE CONTRADIGAM A ÚLTIMA RESPOSTA DO PACIENTE.** Se o paciente disse que algo melhorou, não pergunte se piorou ou continua igual.
* A PERGUNTA DEVE SER DIRECIONADA e concisa.
* Varie a forma de fazer as perguntas.

Exemplos de perguntas adequadas após o paciente relatar melhora:
* 'Que bom que a coceira melhorou! Você notou alguma outra mudança na sua pele?'
* 'Considerando que a vermelhidão está melhor, como você avalia o seu conforto geral hoje?'
* 'Além da melhora na coceira, há alguma outra área que você percebeu diferente?'

Use um tom atencioso e encorajador. """
    response = client.models.generate_content(model=modelo_nome, contents=prompt)
    return response.text

# --- Início da Conversa ---
if "tratamento" in tipo_tratamento.lower():
    mensagem_boas_vindas = f"MonitorIA: Olá! Sou seu assistente virtual para o acompanhamento do seu {tipo_tratamento} com {artigo_tratamento} {nome_profissional_final}."
elif tipo_tratamento.lower() == 'dermatologia':
    mensagem_boas_vindas = f"MonitorIA: Olá! Sou seu assistente virtual para o acompanhamento do seu tratamento de {tipo_tratamento} com {artigo_tratamento} {nome_profissional_final}."
else:
    mensagem_boas_vindas = f"MonitorIA: Olá! Sou seu assistente virtual para o acompanhamento do seu tratamento {tipo_tratamento} com {artigo_tratamento} {nome_profissional_final}."

print(mensagem_boas_vindas)
nome_paciente = input("MonitorIA: Qual o seu nome? ")
print(f"MonitorIA: Olá, {nome_paciente}!")
resposta_autorizacao = input(f"MonitorIA: Para que eu possa enviar suas atualizações e a foto (se desejar compartilhar) para {nome_profissional_final}, você autoriza este compartilhamento? (Sim/Não) ")
autorizacao_ok = verificar_resposta(resposta_autorizacao, palavras_afirmativas, palavras_negativas)
if autorizacao_ok is False:
    print("MonitorIA: O compartilhamento das informações foi cancelado. A conversa será encerrada.")
else:
    if autorizacao_ok is True:
        autorizacao_compartilhamento = True
        autorizacao_concedida = True
        print("MonitorIA: Obrigado pela sua autorização.")
        print(f"MonitorIA: Suas atualizações serão acompanhadas {'pelo' if tratamento_desejado.lower() == 'dr.' else 'pela'} {nome_profissional_final}.")
    else:
        print("MonitorIA: Resposta inválida para autorização. A conversa será encerrada.")

if autorizacao_concedida:
    historico_atualizacoes.setdefault(nome_paciente, [])
    primeira_atualizacao = input("MonitorIA: Como está se sentindo hoje? ")
    historico_atualizacoes[nome_paciente].append(primeira_atualizacao)

    # Analisar a primeira atualização
    sentimento_negativo = any(palavra in primeira_atualizacao.lower() for palavra in ["mal", "pior", "dor", "coçando muito", "desconfortável"])

    if sentimento_negativo:
        if tratamento_desejado.lower() == 'dr.':
            print(f"MonitorIA: Sinto muito que você não esteja se sentindo bem, {nome_paciente}. O Dr. {nome_profissional} será notificado sobre isso.")
        elif tratamento_desejado.lower() == 'dra.':
            print(f"MonitorIA: Sinto muito que você não esteja se sentindo bem, {nome_paciente}. A Dra. {nome_profissional} será notificada sobre isso.")
        else:
            print(f"MonitorIA: Sinto muito que você não esteja se sentindo bem, {nome_paciente}. {nome_profissional_final} será notificado sobre isso.") # Caso não haja tratamento especificado
        piora_detectada = True

    # Fazer as perguntas padrão
    respostas_padrao = []
    for pergunta in perguntas_padrao:
        resposta = input(f"MonitorIA: {pergunta} ")
        historico_atualizacoes[nome_paciente].append(resposta)
        respostas_padrao.append(resposta)

    # Pergunta sobre a foto (feita uma vez após as perguntas padrão)
    if not enviou_foto:
        quer_enviar_foto = input("MonitorIA: Gostaria de compartilhar uma foto da área afetada para ajudar na avaliação? (Sim/Não) ")
        compartilhar_foto = verificar_resposta(quer_enviar_foto, palavras_afirmativas, palavras_negativas)
        if compartilhar_foto is True:
            ajuda_link = input("MonitorIA: Para compartilhar uma foto, você pode fazer o upload em um serviço como Google Drive, Imgur ou Dropbox e colar o link aqui. Se precisar de ajuda, digite 'ajuda link'. ")
            if ajuda_link.lower() == "ajuda link":
                print("MonitorIA: Para obter o link no Google Drive:")
                print("1. Faça o upload da foto.")
                print("2. Clique com o botão direito na foto.")
                print("3. Selecione 'Compartilhar'.")
                print("4. Em 'Acesso geral', selecione 'Qualquer pessoa com o link'.")
                print("5. Clique em 'Copiar link' e cole aqui.")
                print("Para Imgur:")
                print("1. Faça o upload da foto.")
                print("2. O link da imagem estará disponível após o upload. Copie e cole aqui.")
                print("Para Dropbox:")
                print("1. Faça o upload da foto.")
                print("2. Clique em 'Compartilhar' ao lado da foto.")
                print("3. Selecione 'Criar e copiar link'.")
                print("4. Cole o link aqui.")
                link_foto = input("MonitorIA: Por favor, cole o link da foto aqui: ")
            else:
                link_foto = ajuda_link
            if link_foto:
                historico_atualizacoes[nome_paciente].append(f"Foto enviada pelo paciente (link): {link_foto.strip()}")
                print(f"MonitorIA: Obrigado, {nome_paciente}! O link da foto será enviado ao {nome_profissional_final}.")
                enviou_foto = True
            else:
                print("MonitorIA: Tudo bem. Se mudar de ideia, pode tentar novamente mais tarde.")
                enviou_foto = True # Mesmo que não envie, para não perguntar novamente
        elif compartilhar_foto is False:
            print("MonitorIA: Entendo. Por favor, continue descrevendo suas atualizações.")
            enviou_foto = True
        else:
            print("MonitorIA: Resposta inválida sobre o compartilhamento da foto.")
            enviou_foto = True

   # Início do loop de conversa para obter mais atualizações
    ultima_pergunta_monitoria = "" # Inicializa para evitar erros na primeira iteração do loop
    continuar_conversa = True
    num_interacoes = 0
    max_interacoes = 3
    primeira_conversa = False # Define como False aqui, após a fase inicial
    while continuar_conversa and num_interacoes < max_interacoes:
        num_interacoes += 1

        houve_piora_agora = verificar_piora(nome_paciente, historico_atualizacoes)
        if houve_piora_agora and not piora_detectada:
            print(f"\nMonitorIA: {nome_paciente}, você mencionou um aumento significativo na vermelhidão. Vou notificar {'o' if tratamento_desejado.lower() == 'dr.' else 'a'} {nome_profissional_final} sobre essa mudança. ")
            corpo_email_piora = f"O paciente {nome_paciente} reportou um aumento significativo na vermelhidão."
            enviar_notificacao_email(email_profissional, "Alerta de Piora na Condição do Paciente", corpo_email_piora)
            piora_detectada = True # Para evitar notificações repetidas

        proxima_pergunta = gerar_proxima_interacao(nome_paciente, historico_atualizacoes,
                                                                     perguntas_padrao, ultima_pergunta_monitoria)
        print(f"\nMonitorIA: {proxima_pergunta}")
        ultima_pergunta_monitoria = f"MonitorIA: {proxima_pergunta}"
        resposta_paciente = input(f"{nome_paciente}: ")
        historico_atualizacoes[nome_paciente].append(resposta_paciente)

        # Verificar se o paciente quer encerrar a conversa (exemplo: "tchau", "fim", etc.)
        if any(palavra in resposta_paciente.lower() for palavra in ["tchau", "fim", "encerrar", "adeus"]):
            continuar_conversa = False
            print("MonitorIA: Agradeço o seu contato. Se precisar, pode me chamar novamente.")

    # --- Fim do loop de conversa ---

    if continuar_conversa: # Se o paciente não encerrou explicitamente
        pergunta_final = "MonitorIA: Há mais alguma coisa que gostaria de me informar hoje?"
        print(f"\n{pergunta_final}")
        resposta_final_paciente = input(f"{nome_paciente}: ")
        historico_atualizacoes[nome_paciente].append(resposta_final_paciente)

        if any(palavra in resposta_final_paciente.lower() for palavra in ["não", "nada", "tudo certo", "obrigado"]):
            print("MonitorIA: Agradeço o seu contato e as suas atualizações.")
            print(f"MonitorIA: Suas informações serão enviadas para {nome_profissional_final}.")
            print("MonitorIA: Desejo uma ótima recuperação!")
        else:
            print("MonitorIA: Obrigado pela informação adicional.")
            print(f"MonitorIA: Suas atualizações completas serão enviadas para {nome_profissional_final}.")
            print("MonitorIA: Desejo uma ótima recuperação!")
    else:
        print(f"MonitorIA: Suas atualizações serão enviadas para {nome_profissional_final}.")
        print("MonitorIA: Desejo uma ótima recuperação!")

    # --- Envio do Resumo para o Profissional (AGORA COM AS PERGUNTAS) ---
    # (O restante do seu código para o envio do e-mail permanece aqui)

    # --- Envio do Resumo para o Profissional (AGORA COM AS PERGUNTAS) ---
    print("\n--- Conteúdo do historico_completo ---")
    print(historico_atualizacoes.get(nome_paciente, []))
    print("--- Fim do conteúdo do historico_completo ---\n")

    corpo_email_resumo = f"Resumo das atualizações do paciente {nome_paciente}:\n\n"
    historico_completo = historico_atualizacoes.get(nome_paciente, [])
    perguntas_e_respostas = []

    # Incluir a primeira atualização (como o paciente estava se sentindo inicialmente)
    if historico_completo:
        perguntas_e_respostas.append(f"MonitorIA (Inicial): Como está se sentindo hoje?")
        perguntas_e_respostas.append(f"{nome_paciente}: {historico_completo[0]}")

    # Incluir as perguntas padrão e suas respostas
    for i, pergunta in enumerate(perguntas_padrao):
        if len(historico_completo) > i + 1:
            perguntas_e_respostas.append(f"MonitorIA: {pergunta}")
            perguntas_e_respostas.append(f"{nome_paciente}: {historico_completo[i + 1]}")

    # Incluir a pergunta sobre a foto e a resposta (se houver link)
    if enviou_foto and any("Foto enviada pelo paciente (link):" in item for item in historico_completo):
        perguntas_e_respostas.append("MonitorIA: Gostaria de compartilhar uma foto da área afetada para ajudar na avaliação?")
        for item in historico_completo:
            if "Foto enviada pelo paciente (link):" in item:
                link_parte = item.split(': ')[1]
                perguntas_e_respostas.append(f"{nome_paciente}: Sim (link: {link_parte})")
                break # Para não repetir o link se houver outras menções

    # Incluir as perguntas geradas dinamicamente e suas respostas
    num_interacoes_adicionais = len(historico_completo) - 2 - len(perguntas_padrao) # Ajuste para contar as interações Gemini
    if num_interacoes_adicionais > 0:
        indice_interacao_gemini = 2 + len(perguntas_padrao) # Onde as interações Gemini começam no histórico
        for i in range(num_interacoes_adicionais):
            if len(historico_completo) > indice_interacao_gemini + i * 2 - 1 and len(historico_completo) > indice_interacao_gemini + i * 2:
                pergunta_gemini = historico_completo[indice_interacao_gemini + i * 2 - 1]
                resposta_gemini = historico_completo[indice_interacao_gemini + i * 2]
                if "MonitorIA:" in pergunta_gemini:
                    perguntas_e_respostas.append(pergunta_gemini)
                    perguntas_e_respostas.append(f"{nome_paciente}: {resposta_gemini}")


    corpo_email_resumo += "\n".join(perguntas_e_respostas)

    assunto_email_resumo = f"Atualizações Detalhadas do Paciente {nome_paciente}"

    print(f"Destinatário do email de resumo: {email_profissional}")
    print(f"Assunto do email de resumo: {assunto_email_resumo}")
    print(f"Corpo do email de resumo:\n{corpo_email_resumo}")

    enviar_notificacao_email(email_profissional, assunto_email_resumo, corpo_email_resumo)
    print(f"MonitorIA: As atualizações detalhadas de {nome_paciente} foram enviadas para {nome_profissional_final}.")

MonitorIA: Olá! Sou seu assistente virtual para o acompanhamento do seu tratamento Pós cirurgico com  Dra.  Jessica.
MonitorIA: Qual o seu nome? eds
MonitorIA: Olá, eds!
MonitorIA: Para que eu possa enviar suas atualizações e a foto (se desejar compartilhar) para Dra.  Jessica, você autoriza este compartilhamento? (Sim/Não) s
MonitorIA: Obrigado pela sua autorização.
MonitorIA: Suas atualizações serão acompanhadas pela Dra.  Jessica.
MonitorIA: Como está se sentindo hoje? bem
MonitorIA: Como está a dor? de boa
MonitorIA: Já iniciou as drenagens? sim
MonitorIA: Está fazendo o uso da cinta em tempo integral? sim
MonitorIA: Houve alguma alteração na textura da pele ao redor da cicatriz? nao
MonitorIA: Gostaria de compartilhar uma foto da área afetada para ajudar na avaliação? (Sim/Não) sim
MonitorIA: Para compartilhar uma foto, você pode fazer o upload em um serviço como Google Drive, Imgur ou Dropbox e colar o link aqui. Se precisar de ajuda, digite 'ajuda link'. https://drive.google.com